In [ ]:
import datetime
import os.path

import cftime
import numpy as np
import pygetm.legacy
import pygetm.input
#pygetm.input.debug_nc_reads()
import cartopy.crs as ccrs

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
igotm_data = next(filter(os.path.isdir, ('/server/data', '../../../igotm/data')))

northsea_path = os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc')  # path to legacy GETM North Sea test case
era_path = os.path.join(igotm_data, 'ERA-interim/20??.nc')              # path to ERA NetCDF file with t2m variable
gebco_path = os.path.join(igotm_data, 'GEBCO/GEBCO_2022.nc')            # path to GEBCO NetCDF file with elevation variable
tpxo_path = os.path.join(igotm_data, 'TPXO9/h_m2_tpxo9_atlas_30.nc')    # path to TPXO9 NetCDF file with hRe/hIm variables
woa_path = os.path.join(igotm_data, 'WOA2018/woa.nc')                   # path to WOA NetCDF file with t_an variable

map_proj = ccrs.PlateCarree()                                      # map projection to use for plotting

def era_preprocess(ds):
    return ds.isel(time=slice(4, -4))

In [ ]:
# Load North Sea domain - we'll interpolate to it later
domain = pygetm.legacy.domain_from_topo(northsea_path, nlev=1)
domain.initialize(1)

# Obtain temperature variable - this should not read the data yet
t2m = pygetm.input.from_nc(era_path, "t2m", preprocess=era_preprocess)

In [ ]:
# Plot first time of air temperature - full domain
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
t2m0 = t2m[..., 0]
t2m0.plot()
ax.coastlines()

In [ ]:
# Limit domain to the North Sea region
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
t2m0_clip = pygetm.input.limit_region(
    t2m0,
    domain.lon.min(),
    domain.lon.max(),
    domain.lat.min(),
    domain.lat.max(),
    periodic_lon=True,
)
t2m0_clip.plot()
ax.coastlines()

In [ ]:
# Interpolate to actual North Sea grid
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
t2m0_sip = pygetm.input.horizontal_interpolation(
    t2m0_clip, domain.lon[1::2, 1::2], domain.lat[1::2, 1::2]
)
t2m0_sip.plot(x="longitude_", y="latitude_")
ax.coastlines()

In [ ]:
# Test temporal interpolation (full domain)
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
t2m_tip = pygetm.input.temporal_interpolation(t2m)
t2m_tip.data.update(cftime.datetime(2016, 5, 1))
t2m_tip.plot()
ax.coastlines()

In [ ]:
# Full treatment of air temperature as in model:
# - open NetCDF
# - determine extents of region of interest
# - interpolate temporally
# - interpolate spatially
t2m_clip = pygetm.input.limit_region(
    t2m,
    domain.lon.min(),
    domain.lon.max(),
    domain.lat.min(),
    domain.lat.max(),
    periodic_lon=True,
)
t2m_clip_sip = pygetm.input.horizontal_interpolation(
    t2m_clip, domain.lon[1::2, 1::2], domain.lat[1::2, 1::2]
)
t2m_clip_sip_tip = pygetm.input.temporal_interpolation(t2m_clip_sip) - 273.15

# Set up plot of initial field - we'll animate it later
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
start = cftime.datetime(2016, 5, 1)
t2m_clip_sip_tip.variable.data.update(start)
pc = t2m_clip_sip_tip.plot(x="longitude_", y="latitude_", vmin=5, vmax=20)
ax.coastlines()

In [ ]:
# Update air temperature in 15 minute steps over a ten-day period
for i in range(24 * 4 * 10):
    dt = start + datetime.timedelta(seconds=i * 900)
    t2m_clip_sip_tip.data.update(dt)
    pc.set_array(t2m_clip_sip_tip.values.ravel())
    ax.set_title(dt)
    fig.canvas.draw()

In [ ]:
# Get GEBCO bathymetry
elev = pygetm.input.from_nc(gebco_path, "elevation")
elev = pygetm.input.limit_region(
    elev,
    domain.lon.min(),
    domain.lon.max(),
    domain.lat.min(),
    domain.lat.max(),
    periodic_lon=True,
)
elev = pygetm.input.horizontal_interpolation(elev, domain.lon, domain.lat)

fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
elev.plot(x="lon_", y="lat_", vmin=-1000, vmax=0)
ax.coastlines()

In [ ]:
# Get M2 tidal amplitude on T grid
def tpxo_preprocess(ds):
    return ds.set_coords(("lat_z", "lon_z"))

m2_re = domain.T.array()
m2_re.set(pygetm.input.from_nc(tpxo_path, "hRe", preprocess=tpxo_preprocess))

m2_im = domain.T.array()
m2_im.set(pygetm.input.from_nc(tpxo_path, "hIm", preprocess=tpxo_preprocess))

amp = np.sqrt(m2_re ** 2 + m2_im ** 2)
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
amp.plot()
ax.coastlines()

In [ ]:
# Get WOA temperature
t_an = pygetm.input.from_nc(woa_path, "t_an")
t_an = pygetm.input.limit_region(
    t_an,
    domain.lon.min(),
    domain.lon.max(),
    domain.lat.min(),
    domain.lat.max(),
    periodic_lon=True,
)
mask = np.isnan(t_an.values)
t_an = t_an.where(~mask, 0)
t_an_ip = pygetm.input.horizontal_interpolation(t_an, domain.lon, domain.lat, mask=mask)

fig, (ax1, ax2) = matplotlib.pyplot.subplots(
    ncols=2, subplot_kw={"projection": map_proj}, figsize=(12, 4)
)
t_an[..., 0].plot(ax=ax1)
t_an_ip[..., 0].plot(ax=ax2, x="lon_", y="lat_")

In [ ]:
# Get u10 and v10 on T grid (spatial limit, temporal interpolation, spatial interpolation all handled in the background)
u10 = domain.T.array()
v10 = domain.T.array()
u10.set(pygetm.input.from_nc(era_path, "u10", preprocess=era_preprocess))
v10.set(pygetm.input.from_nc(era_path, "v10", preprocess=era_preprocess))

u10s = u10[::2, ::2]
v10s = v10[::2, ::2]

# Set up plot of initial field - we'll animate it later
fig, ax = matplotlib.pyplot.subplots(subplot_kw={"projection": map_proj})
start = cftime.datetime(2016, 1, 1)
domain.input_manager.update(start)
Q = ax.quiver(
    u10.grid.lon[::2, ::2],
    u10.grid.lat[::2, ::2],
    u10s,
    v10s,
    np.sqrt(u10s ** 2 + v10s ** 2),
    scale=500,
)
ax.coastlines()

In [ ]:
# Update wind vectors in 1 hour steps over a ten-day period
for i in range(24 * 10):
    dt = start + datetime.timedelta(seconds=(i + 1) * 3600)
    domain.input_manager.update(dt)
    Q.set_UVC(u10s, v10s, np.sqrt(u10s ** 2 + v10s ** 2))
    ax.set_title(dt)
    fig.canvas.draw()